In [1]:
!ls

config.py   embedding.py  images	     README.md	skipthoughts.py
decoder.py  generate.py   NewNotebook.ipynb  search.py


In [1]:
import generate

AttributeError: 'module' object has no attribute 'FLAG_CPU_MODE'

In [39]:
!ls /home/jack/AI_Writer-master/Lasagne/lasagne

conftest.py  __init__.pyc	objectives.py	   tests	      utils.py
__init__.py  layers		random.py	   theano_extensions
init.py      nonlinearities.py	regularization.py  updates.py


In [43]:
import caffe

ImportError: No module named caffe

In [42]:
import sys
sys.path.insert(0, "/home/jack/AI_Writer-master")
import lasagne
from lasagne import nonlinearities

ImportError: cannot import name nonlinearities

In [23]:
!locate theano/tensor/signal/downsample

/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.py
/usr/local/lib/python2.7/dist-packages/theano/tensor/signal/downsample.pyc


In [17]:
!ls /usr/local/lib/python2.7/dist-packages/theano/tensor/signal/

conv.py   downsample.py   __init__.py	pool.py   tests
conv.pyc  downsample.pyc  __init__.pyc	pool.pyc


In [19]:
#/usr/local/lib/python2.7/dist-packages/
import sys
sys.path.insert(0, "/usr/local/lib/python2.7/dist-packages")
import theano
sys.path.insert(0, "/usr/local/lib/python2.7/dist-packages/theano/tensor/signal")
#from theano.tensor.signal import downsample
from theano import tensor
from theano.tensor import signal
sys.path.insert(0, "/usr/local/lib/python2.7/dist-packages/theano/tensor/signal")
from signal import downsample

ImportError: cannot import name downsample

In [ ]:
# %load generate.py
"""
Story generation
"""
import cPickle as pkl
import numpy
import copy
import sys
import skimage.transform

import skipthoughts
import decoder
import embedding

import config


import lasagne
from lasagne.layers import InputLayer, DenseLayer, NonlinearityLayer, DropoutLayer
from lasagne.layers import MaxPool2DLayer as PoolLayer
from lasagne.nonlinearities import softmax
from lasagne.utils import floatX
if not config.FLAG_CPU_MODE:
    from lasagne.layers.corrmm import Conv2DMMLayer as ConvLayer

from scipy import optimize, stats
from collections import OrderedDict, defaultdict, Counter
from numpy.random import RandomState
from scipy.linalg import norm

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True


def story(z, image_loc, k=100, bw=50, lyric=False):
    """
    Generate a story for an image at location image_loc
    """
    # Load the image
    rawim, im = load_image(image_loc)

    # Run image through convnet
    feats = compute_features(z['net'], im).flatten()
    feats /= norm(feats)

    # Embed image into joint space
    feats = embedding.encode_images(z['vse'], feats[None,:])

    # Compute the nearest neighbours
    scores = numpy.dot(feats, z['cvec'].T).flatten()
    sorted_args = numpy.argsort(scores)[::-1]
    sentences = [z['cap'][a] for a in sorted_args[:k]]

    print 'NEAREST-CAPTIONS: '
    for s in sentences[:5]:
        print s
    print ''

    # Compute skip-thought vectors for sentences
    svecs = skipthoughts.encode(z['stv'], sentences, verbose=False)

    # Style shifting
    shift = svecs.mean(0) - z['bneg'] + z['bpos']

    # Generate story conditioned on shift
    passage = decoder.run_sampler(z['dec'], shift, beam_width=bw)
    print 'OUTPUT: '
    if lyric:
        for line in passage.split(','):
            if line[0] != ' ':
                print line
            else:
                print line[1:]
    else:
        print passage


def load_all():
    """
    Load everything we need for generating
    """
    print config.paths['decmodel']

    # Skip-thoughts
    print 'Loading skip-thoughts...'
    stv = skipthoughts.load_model(config.paths['skmodels'],
                                  config.paths['sktables'])

    # Decoder
    print 'Loading decoder...'
    dec = decoder.load_model(config.paths['decmodel'],
                             config.paths['dictionary'])

    # Image-sentence embedding
    print 'Loading image-sentence embedding...'
    vse = embedding.load_model(config.paths['vsemodel'])

    # VGG-19
    print 'Loading and initializing ConvNet...'

    if config.FLAG_CPU_MODE:
        sys.path.insert(0, config.paths['pycaffe'])
        import caffe
        caffe.set_mode_cpu()
        net = caffe.Net(config.paths['vgg_proto_caffe'],
                        config.paths['vgg_model_caffe'],
                        caffe.TEST)
    else:
        net = build_convnet(config.paths['vgg'])

    # Captions
    print 'Loading captions...'
    cap = []
    with open(config.paths['captions'], 'rb') as f:
        for line in f:
            cap.append(line.strip())

    # Caption embeddings
    print 'Embedding captions...'
    cvec = embedding.encode_sentences(vse, cap, verbose=False)

    # Biases
    print 'Loading biases...'
    bneg = numpy.load(config.paths['negbias'])
    bpos = numpy.load(config.paths['posbias'])

    # Pack up
    z = {}
    z['stv'] = stv
    z['dec'] = dec
    z['vse'] = vse
    z['net'] = net
    z['cap'] = cap
    z['cvec'] = cvec
    z['bneg'] = bneg
    z['bpos'] = bpos

    return z

def load_image(file_name):
    """
    Load and preprocess an image
    """
    MEAN_VALUE = numpy.array([103.939, 116.779, 123.68]).reshape((3,1,1))
    image = Image.open(file_name)
    im = numpy.array(image)

    # Resize so smallest dim = 256, preserving aspect ratio
    if len(im.shape) == 2:
        im = im[:, :, numpy.newaxis]
        im = numpy.repeat(im, 3, axis=2)
    h, w, _ = im.shape
    if h < w:
        im = skimage.transform.resize(im, (256, w*256/h), preserve_range=True)
    else:
        im = skimage.transform.resize(im, (h*256/w, 256), preserve_range=True)

    # Central crop to 224x224
    h, w, _ = im.shape
    im = im[h//2-112:h//2+112, w//2-112:w//2+112]

    rawim = numpy.copy(im).astype('uint8')

    # Shuffle axes to c01
    im = numpy.swapaxes(numpy.swapaxes(im, 1, 2), 0, 1)

    # Convert to BGR
    im = im[::-1, :, :]

    im = im - MEAN_VALUE
    return rawim, floatX(im[numpy.newaxis])

def compute_features(net, im):
    """
    Compute fc7 features for im
    """
    if config.FLAG_CPU_MODE:
        net.blobs['data'].reshape(* im.shape)
        net.blobs['data'].data[...] = im
        net.forward()
        fc7 = net.blobs['fc7'].data
    else:
        fc7 = numpy.array(lasagne.layers.get_output(net['fc7'], im,
                                                    deterministic=True).eval())
    return fc7

def build_convnet(path_to_vgg):
    """
    Construct VGG-19 convnet
    """
    net = {}
    net['input'] = InputLayer((None, 3, 224, 224))
    net['conv1_1'] = ConvLayer(net['input'], 64, 3, pad=1)
    net['conv1_2'] = ConvLayer(net['conv1_1'], 64, 3, pad=1)
    net['pool1'] = PoolLayer(net['conv1_2'], 2)
    net['conv2_1'] = ConvLayer(net['pool1'], 128, 3, pad=1)
    net['conv2_2'] = ConvLayer(net['conv2_1'], 128, 3, pad=1)
    net['pool2'] = PoolLayer(net['conv2_2'], 2)
    net['conv3_1'] = ConvLayer(net['pool2'], 256, 3, pad=1)
    net['conv3_2'] = ConvLayer(net['conv3_1'], 256, 3, pad=1)
    net['conv3_3'] = ConvLayer(net['conv3_2'], 256, 3, pad=1)
    net['conv3_4'] = ConvLayer(net['conv3_3'], 256, 3, pad=1)
    net['pool3'] = PoolLayer(net['conv3_4'], 2)
    net['conv4_1'] = ConvLayer(net['pool3'], 512, 3, pad=1)
    net['conv4_2'] = ConvLayer(net['conv4_1'], 512, 3, pad=1)
    net['conv4_3'] = ConvLayer(net['conv4_2'], 512, 3, pad=1)
    net['conv4_4'] = ConvLayer(net['conv4_3'], 512, 3, pad=1)
    net['pool4'] = PoolLayer(net['conv4_4'], 2)
    net['conv5_1'] = ConvLayer(net['pool4'], 512, 3, pad=1)
    net['conv5_2'] = ConvLayer(net['conv5_1'], 512, 3, pad=1)
    net['conv5_3'] = ConvLayer(net['conv5_2'], 512, 3, pad=1)
    net['conv5_4'] = ConvLayer(net['conv5_3'], 512, 3, pad=1)
    net['pool5'] = PoolLayer(net['conv5_4'], 2)
    net['fc6'] = DenseLayer(net['pool5'], num_units=4096)
    net['fc7'] = DenseLayer(net['fc6'], num_units=4096)
    net['fc8'] = DenseLayer(net['fc7'], num_units=1000, nonlinearity=None)
    net['prob'] = NonlinearityLayer(net['fc8'], softmax)

    print 'Loading parameters...'
    output_layer = net['prob']
    model = pkl.load(open(path_to_vgg))
    lasagne.layers.set_all_param_values(output_layer, model['param values'])

    return net




In [ ]:
Collecting https://github.com/theano/theano/archive/master.zip
  Downloading https://github.com/theano/theano/archive/master.zip (13.4MB)
    100% |████████████████████████████████| 13.4MB 53kB/s 
Collecting numpy>=1.9.1 (from Theano==1.0.1+unknown)
  Downloading numpy-1.14.0-cp27-cp27mu-manylinux1_x86_64.whl (16.9MB)
    100% |████████████████████████████████| 16.9MB 54kB/s 
Collecting scipy>=0.14 (from Theano==1.0.1+unknown)
  Downloading scipy-1.0.0-cp27-cp27mu-manylinux1_x86_64.whl (46.7MB)
    100% |████████████████████████████████| 46.7MB 18kB/s 
Requirement already up-to-date: six>=1.9.0 in /home/jack/anaconda2/lib/python2.7/site-packages (from Theano==1.0.1+unknown)
Installing collected packages: numpy, scipy, Theano
  Found existing installation: numpy 1.13.3
    Uninstalling numpy-1.13.3:
      Successfully uninstalled numpy-1.13.3
  Found existing installation: scipy 0.19.1
    Uninstalling scipy-0.19.1:
      Successfully uninstalled scipy-0.19.1
  Found existing installation: Theano 0.9.0
    Uninstalling Theano-0.9.0:
      Successfully uninstalled Theano-0.9.0
  Running setup.py install for Theano ... done
Successfully installed Theano-1.0.1+unknown numpy-1.14.0 scipy-1.0.0
jack@jack-desktop:~/iso$ 


In [23]:
"""
A simple TKinter GUI to enter data into a given table in a database.

This program will build a small sample table into a given database
and then build a simple TKinter window with label and entry widgets
for each column in the table.
"""
import sqlite3
import tkinter as tk
from tkinter import N, S, E, W
from tkinter import TOP, BOTTOM, LEFT, RIGHT, END, ALL


def main():
    """Main function for demo."""
    # define some variables for demo only.
    # In full app, these will come programmatically.
    db = 'foo.sqlite'
    tbl = 'bar'
    columns = 'code text', 'keywords text'
    create_table(db, tbl, *columns)

    root = tk.Tk()
    demo_window = EntryWindow(root, *[db, tbl])
    root.mainloop()


# create a sample table for demo purposes only.
# in full app the database schema would be created elsewhere.
def create_table(database, table, *col_defs):
    """
    Insert a simple table into sqlite3 database.

    Args:
        database (string):
        Name of database function will be creating table in.

        table (string):
        Name of table to be created in given database.

        *col_defs (tuple of strings):
        A tuple of strings containing the SQL column definitions for the
        given table being created in given database.
    """
    stmnt = (('CREATE TABLE IF NOT EXISTS {}('+('{},'*len(col_defs))[:-1]+');')
             .format(table, *col_defs))
    with sqlite3.connect(database) as conn:
        c = conn.cursor()
        #c.execute('drop table if exists {};'.format(table))
        c.execute(stmnt)
        conn.commit()


class EntryWindow(tk.Frame):

    """
    Provides a simple data entry window for a given table in given database.

    Automatically generates labels and entry fields based on the column
    headers for the given table.  Provides a button to submit the row of data
    into the table and a button to close window.
    """

    def __init__(self, master=None, *args):
        tk.Frame.__init__(self, master)
        self.master = master
        self.database = args[0]
        self.table = args[1]
        self.init_window()

    def init_window(self):
        """Build the entry window."""
        self.master.title('Enter data into {}'.format(self.table.upper()))
        self.grid(column=200, row=200, sticky=(N, W, E, S), padx=10, pady=5)
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)

        def get_col_names(self):
            """Retrieve column names for given table in database."""
            with sqlite3.connect(self.database) as conn:
                c = conn.cursor()
                c.execute("PRAGMA table_info('{}')".format(self.table))
                # Currently only uses one value from returned tuple.
                # TODO: Add logic to utilize type, not null and PK fields.
                self.col_names = [x[1] for x in c.fetchall()]
            return self.col_names

        self.column_names = get_col_names(self)

        # Add a label and entry box for each column in table.
        # TODO: Add functionality to gray out primary key fields where the
        #       database should be assigning the next value.
        # TODO: Add some validation logic.
        self.item_entry = []
        for item in self.column_names:
            num = len(self.item_entry)
            # print(num, ' --> '+item)
            tk.Label(self, text=item).grid(row=num, column=0,
                                           pady=1, sticky=E)
            self.item_entry.append(tk.Entry(self))
            self.item_entry[num].grid(row=num, column=1, pady=1, padx=5)

        def add_item(self):
            """Get entries from input fields and insert into database table."""
            entries = [e.get() for e in self.item_entry]
            # Build the SQL statement
            stmnt = ('insert into {0}({1}) values ({2})'
                     .format(self.table, ','.join(self.column_names),
                             ':'+',:'.join(self.column_names)))
            # print(stmnt, entries)
            with sqlite3.connect(self.database) as conn:
                c = conn.cursor()
                c.execute(stmnt, entries)
                conn.commit()
            clear_fields(self)

        def clear_fields(self):
            """Clear fields of entry windo and return focus to first field."""
            for e in self.item_entry:
                e.delete(0, END)
            self.item_entry[0].focus()

        # Add button to submit user inputs into database.
        submit_button = tk.Button(self, text='Add Item', width=8,
                                  command=lambda: add_item(self))
        submit_button.grid(row=3, column=0, sticky=E, pady=10, padx=1)

        # Add a cancel button which closes window.
        quit_button = tk.Button(self, text='Cancel', width=8,
                                command=self.quit)
        quit_button.grid(row=3, column=1, sticky=W, pady=10, padx=1)

if __name__ == '__main__':
    main()


KeyboardInterrupt: 

In [2]:
!ls

config.py     embedding.pyc	      images		 search.pyc
config.pyc    foo.sqlite	      NewNotebook.ipynb  skipthoughts.py
decoder.py    generate.py	      README.md		 skipthoughts.pyc
decoder.pyc   generate.pyc	      requirements.txt
embedding.py  GenIm.pynotebook.ipynb  search.py


In [25]:
import sqlite3
conn = sqlite3.connect("foo.sqlite")
c = conn.cursor()
rows = c.execute("SELECT * FROM bar")
for row in rows:
    print(row)

(u'66666666', u'ggggggggg')
(u'12121212', u'zzzzzzzz')
(u'5656565', u'565656')
(u'new', u'added')


In [ ]:
import sqlite3
conn = sqlite3.connect("foo.sqlite")
c = conn.cursor()
rows = c.execute("SELECT * FROM bar")
for row in rows:
    print(row)

In [24]:
!ls

config.py     embedding.pyc	      images		 search.pyc
config.pyc    foo.sqlite	      NewNotebook.ipynb  skipthoughts.py
decoder.py    generate.py	      README.md		 skipthoughts.pyc
decoder.pyc   generate.pyc	      requirements.txt	 snippet.db
embedding.py  GenIm.pynotebook.ipynb  search.py


# Below is a preloaded sqlite database full of snippets
Run cell<br />
enter search term try the word 'create' for example: <br />
Searches may also be created with bolean example: <br />
create + fts4 <br /> <br />
press enter key:<br /> 
###    Clicking cell to search will not work

In [27]:
import sqlite3
conn = sqlite3.connect("snippet.db")
c = conn.cursor()
print ("Entry must be made by enter key NOT a mouse 'click'")
search = raw_input("SEARCH TERM :")
rows = c.execute("SELECT rowid, * FROM snippet WHERE snippet MATCH ?", (search,))
for row in rows:
    print row[2]

Entry must be made by enter key NOT a mouse 'click'
SEARCH TERM :create + fts4

import sqlite3
import re
import sys
import time
database = "test.db"
#database = "junk2.db"
conn = sqlite3.connect(database)
conn.text_factory = lambda x: unicode(x, "utf-8", "ignore")
c = conn.cursor()
c.execute(""
CREATE VIRTUAL TABLE IF NOT EXISTS ipynb 
USING FTS4(file, content, description);
"")
conn.commit()
conn.close()
conn = sqlite3.connect(database)
c = conn.cursor()
count=1
titlelist = "ipynb-clean.list"
titles = open(titlelist,"r")
for title in titles.readlines():
    filename = os.path.basename(title)
    # Use for debug print filename,":"
    description = filename
    description = description.replace("-", " ")
    description = description.replace("_", " ")
    description = description.replace(".ipynb", " ")
    description = re.sub("([a-z])([A-Z])","\g<1> \g<2>",description)
    title = title.replace("
", "")
    
    dt=time.ctime(os.path.getctime(title))
    dt=str(dt)
    #dt = dt.repla